In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose, ZeroPadding2D, Cropping2D
from tensorflow.keras.models import Model
from shutil import copyfile, rmtree
from timeit import default_timer as timer

In [ ]:
# Вспомогательная функция для доступа к файлам относительно корня директория с данными.
INPUT_ROOT = "../input/gtsrb-german-traffic-sign"
def from_input(path):
    return os.path.join(INPUT_ROOT, path)

In [ ]:
# Загружаем таблицу с данными о данных.
train_info = pd.read_csv(from_input("Train.csv"))
train_info.head()

In [ ]:
# Посмотрим как выглядят наши данные.
train_info.describe()

In [ ]:
# сколько примеров в каждом из классов
train_info.groupby('ClassId')['ClassId'].count()

In [ ]:
test_info =  pd.read_csv(from_input("Test.csv"))
test_info.head()

In [ ]:
test_info.describe()

In [ ]:
# сколько примеров в каждом из классов
test_info.groupby('ClassId')['ClassId'].count()

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Показываем изображения в сетке 6х8.
nrows = 8
ncols = 6

pic_offset = 0 # Чтобы итерировать по изображениям каждый раз когда запустим код ниже.

In [ ]:
def show_images(offset):
    fig = plt.gcf()
    fig.set_size_inches(ncols*3, nrows*3)

    for i in range(43):
        # subplot индексы начинаются с 1
        sp = plt.subplot(nrows, ncols, i + 1)
        sp.axis('Off')
        subdir = os.path.join(from_input('train'), str(i))
        files = os.listdir(subdir)
        img_path = os.path.join(subdir, files[offset % len(files)])
        img = mpimg.imread(img_path)
        #print(img.shape)
        plt.imshow(img)

    plt.show()

In [ ]:
show_images(pic_offset)
pic_offset += 1

Загрузка и подготовка данных:

In [ ]:
TARGET_SIZE = (40, 40) # изображения будут изменены до этого размера
FLATTEN_SIZE = TARGET_SIZE[0] * TARGET_SIZE[1] * 3
BATCH_SIZE=300

In [ ]:
paths = train_info['Path'].values
y_train = train_info['ClassId'].values

indices = np.arange(y_train.shape[0])
randgen = random.Random(62)
randgen.shuffle(indices)

paths = paths[indices]
y_train = y_train[indices]
y_train = to_categorical(y_train, 43)

train_data=[]

for i, f in enumerate(paths):
    print('\rLoading data {0:.1f}%...'.format((i / len(paths)) * 100), end = '\r')
    image = load_img(os.path.join(from_input('train'), f.replace('Train/', '')), target_size=TARGET_SIZE)
    train_data.append(img_to_array(image))

print('Data loaded.              ')


In [ ]:
X_train = tf.keras.applications.vgg16.preprocess_input(np.array(train_data))
train_data = None

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow(X_train,
                                    y_train,
                                    batch_size=BATCH_SIZE,
                                    shuffle=True,
                                    seed=17)

train_aug_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                       rotation_range = 18,
                                       width_shift_range = 0.18,
                                       height_shift_range = 0.18,
                                       shear_range = 0.18,
                                       zoom_range = 0.18,
                                       horizontal_flip = False)

train_aug_generator = train_datagen.flow(X_train,
                                    y_train,
                                    batch_size=BATCH_SIZE,
                                    shuffle=True,
                                    seed=17)

In [ ]:
paths = test_info['Path'].values
y_test = test_info['ClassId'].values
y_test = to_categorical(y_test, 43)

test_data=[]

for i, f in enumerate(paths):
    print('\rLoading data {0:.1f}%...'.format((i / len(paths)) * 100), end = '\r')
    image = load_img(os.path.join(from_input('test'), f.replace('Test/', '')), target_size=TARGET_SIZE)
    test_data.append(img_to_array(image))

print('Data loaded.              ')

In [ ]:
X_test = tf.keras.applications.vgg16.preprocess_input(np.array(test_data))
test_data = None

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow(X_test,
                                    y_test,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False,
                                    seed=17)

Некоторые вспомогательные функции:

In [ ]:
def plot(history, plot_acc = True):
    %matplotlib inline

    import matplotlib.image  as mpimg
    import matplotlib.pyplot as plt

    
    loss=history.history['loss']
    epochs=range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, 'r', "Training Loss")
    plt.xlabel('Epoch')
    plt.title('Training loss')
    # validation
    plt.plot(epochs, history.history['val_loss'], 'b', "Validation Loss")

    if plot_acc:
        acc=history.history['acc']
        plt.figure()
        plt.plot(epochs, acc, 'r', "Training Accuracy")
        plt.title('Training accuracy')
        plt.xlabel('Epoch')
        plt.plot(epochs, history.history['val_acc'], 'b', "Validation Accuracy")



In [ ]:
def show_layers(model):
    print('Name\tOutput shape\tActivation\tInitializer')
    for l in model.layers:
        print('{0}({1})\t{2}\t{3}\t{4}'
            .format(l.name,
              l.__class__.__name__,
              l.output_shape,
              l.activation.__name__ if hasattr(l, 'activation') else '<none>',
              l.kernel_initializer.__class__.__name__ if hasattr(l, 'kernel_initializer') else '<none>'))


def custom_summary(model):
    model.summary()
    show_layers(model)

In [ ]:
VERBOSE=1

In [ ]:
def train_model(model, optimizer, epochs, train_generator):
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    start_time = timer()
    history = model.fit_generator(train_generator,
                        epochs=epochs,
                        verbose=VERBOSE,
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='acc', min_delta=0.0001, patience=2)],
                        validation_data=test_generator,
                        steps_per_epoch= round(X_train.shape[0] / BATCH_SIZE))
    end_time = timer()
    
    print('==============================')
    print('Optimizer: ', optimizer.__class__.__name__)
    print('Learning rate: ', optimizer.get_config()['learning_rate'])
    print('Epochs: ', epochs)
    print('==============================')
    print('Trained in {0:.2f} minutes'.format((end_time - start_time) / 60))
    
    acc=history.history['acc'][-1]
    test_acc = model.evaluate_generator(test_generator)[1]
    
    print('Results at the end of training: acc={1:.02f}%, test_acc={2:.02f}%'
          .format(i, acc*100, test_acc*100))

    plot(history)

In [ ]:
def get_classificator_model(base_model, last_output):
    x = Flatten()(last_output)
    x = Dropout(0.33)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.33)(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(43, activation='softmax')(x)           

    model = Model(base_model.input, x)
    return model

Запускаем готовую сеть на наших данных.

**Эксперимент №1**: Реализуем переноса признакового описания.

In [ ]:
IMG_SHAPE = TARGET_SIZE + (3,)
pretrained_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

pretrained_model.summary()

В этом эксперименте не хотим тренировать слои уже обученной сети.

In [ ]:
for layer in pretrained_model.layers:
    layer.trainable = False

Не будем использовать последний блок VGG16, так как он наверное слишком приспособлен для задаче на которой тренировался. Начальные слои обычно представляют более общие признаки.

Тренируем только нашу часть сети. Первую часть VGG16 используем как метод выделения признаков.

In [ ]:
last_layer = pretrained_model.get_layer('block4_conv3')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

optimizer=Adam(learning_rate=0.0001)
epochs=50
train_model(get_classificator_model(pretrained_model, last_output), optimizer, epochs, train_generator)

Получаем сильное переобучение. На тренировочной выборке модель сходится слишком быстро.

**Эксперимент №2**: Попробуем полностью обучать сеть начиная с уже готовых весов начальных слоёв.

In [ ]:
pretrained_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

for layer in pretrained_model.layers:
    layer.trainable = True

In [ ]:
last_layer = pretrained_model.get_layer('block4_conv3')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

optimizer=Adam(learning_rate=0.00001)
epochs=50
train_model(get_classificator_model(pretrained_model, last_output), optimizer, epochs, train_generator)

**Эксперимент №3**: Попробуем полностью обучать сеть начиная случайной подборки весов всей сети.

In [ ]:
deep_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights=None)
for layer in deep_model.layers:
    layer.trainable = True

In [ ]:
last_layer = deep_model.get_layer('block4_conv3')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

optimizer=Adam(learning_rate=0.00005)
epochs=50
train_model(get_classificator_model(deep_model, last_output), optimizer, epochs, train_generator)

Для сравнения, с такими данными запустим самую хорошую модель из педыдущей лабы

In [ ]:
optimizer=Adam(learning_rate=0.00001)
epochs=50

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(256, (7, 7), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(43, activation='softmax')
])

train_model(model, optimizer, epochs, train_generator)

Попробуем ещё улучшить собственную сеть.

In [ ]:
optimizer=Adam(learning_rate=0.00001)
epochs=50

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(256, (7, 7), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.33),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.33),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.33),
    tf.keras.layers.Dense(1024, activation='tanh'),
    tf.keras.layers.Dropout(0.33),
    tf.keras.layers.Dense(43, activation='softmax')
])

train_model(model, optimizer, epochs, train_generator)

In [ ]:
optimizer=Adam(learning_rate=0.00001)
epochs=50

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(256, (7, 7), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.33),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.33),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', input_shape=TARGET_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.33),
    tf.keras.layers.Dense(1024, activation='tanh'),
    tf.keras.layers.Dropout(0.33),
    tf.keras.layers.Dense(43, activation='softmax')
])

train_model(model, optimizer, epochs, train_aug_generator)